In [43]:
import requests
import bs4 as bs

### Data Sources

- Verbatim archives, i.e. debates by time and day http://164.100.47.5/newsite/debatenew/newshow.aspx?arch=245
    - This includes the un edited debates noted down during the conduct by hours of the days
    - We can get what organisations are being talked about and acts/bills
    - People who are speaking
    - when interuptions are happening
    - The number of hours the rajya sabha worked
- Date Wise Full day edited debates http://164.100.47.5/newsite/floor_official_debate/floor_official_debate.aspx
    - Index of everything laid on the table with page indices
    - More concise

In [2]:
url = 'http://164.100.47.5/newsite/floor_official_debate/floor_official_debate.aspx'

In [35]:
def get_soup(url):
    get_resp = requests.get(url)
    soup = bs.BeautifulSoup(get_resp.text)
    return soup

def extract_aspx_variables_from_soup(soup, var):
    return soup.find('input', {'id':var}).attrs['value']

In [38]:
def get_pdf_links(url, session_id):
    get_resp_soup = get_soup(url)
    data = {'ctl00$ContentPlaceHolder1$DropDownList1': session_id,
           '__VIEWSTATE': extract_aspx_variables_from_soup(get_resp_soup, '__VIEWSTATE'),
           '__VIEWSTATEGENERATOR': extract_aspx_variables_from_soup(get_resp_soup, '__VIEWSTATEGENERATOR'),
           '__EVENTVALIDATION': extract_aspx_variables_from_soup(get_resp_soup, '__EVENTVALIDATION'),
           '__EVENTTARGET': 'ctl00$ContentPlaceHolder1$DropDownList1'}
    resp = requests.post(url, data=data)
    print(resp, resp.headers)
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', {'id':'ctl00_ContentPlaceHolder1_GridView4'})
    return table

In [42]:
get_pdf_links(url, '246')

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


<Response [200]> {'Cache-Control': 'private', 'Content-Type': 'text/html; charset=utf-8', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Set-Cookie': 'ASP.NET_SessionId=reshinymxj4k3quguiu3qzoj; path=/; HttpOnly', 'Date': 'Sat, 24 Aug 2019 00:59:59 GMT', 'Content-Length': '22581', 'X-Client-IP': '183.87.120.194'}


<table border="1" cellspacing="0" id="ctl00_ContentPlaceHolder1_GridView4" rules="all" width="80%">
<tr>
<th scope="col"><font face="Verdana" size="2">S.No.</font></th><th align="center" scope="col"><font face="Verdana" size="2">Date</font></th><th align="center" scope="col"><font face="Verdana" size="2">Hindi Version</font></th><th align="center" scope="col"><font face="Verdana" size="2">Floor Version</font></th>
</tr><tr>
<td><font face="Verdana" size="2">
                              1
                            </font></td><td align="center" width="20%"><font face="Verdana" size="2">10/08/2018</font></td><td align="center" width="40%"><font face="Verdana" size="2"><b>
<span id="ctl00_ContentPlaceHolder1_GridView4_ctl02_sp1" style="display:none;"> <a href="%20" id="ctl00_ContentPlaceHolder1_GridView4_ctl02_hp1" target="_blank"><img alt="hindi version" id="ctl00_ContentPlaceHolder1_GridView4_ctl02_img1" src="images/download.png" style="width :30px"/></a>
</span> </b></font></td><td